In [1]:
import torch
import MiniFL as mfl

In [2]:
def run_client(i:int, client: mfl.algorithms.gd.Client):
    for _ in range(100):
        loss = client.send_grad_get_loss()
        # print(f"Client {i}: {loss}")
        client.apply_global_step()
        
def run_master(master: mfl.algorithms.gd.Master):
    for i in range(100):
        print(f"Master: {master.round()}")

In [3]:
from sklearn.datasets import load_svmlight_file

NUM_CLIENTS = 20

data, labels = load_svmlight_file("phishing.txt")
enc_labels = labels.copy()
data_dense = data.todense()

eval_data = (torch.from_numpy(data_dense).to(torch.float32), torch.from_numpy(enc_labels).to(torch.float32)[:, None])
clients_data = [(x, y) for x, y in zip(torch.split(eval_data[0], len(eval_data[0]) // NUM_CLIENTS, dim=0), torch.split(eval_data[1], len(eval_data[1]) // NUM_CLIENTS, dim=0))]

model = torch.nn.Linear(eval_data[0].shape[1], 1, bias=False)
loss_fn = torch.nn.BCEWithLogitsLoss()

master, clients = mfl.algorithms.gd.get_master_and_clients(
    lr=200,
    clients_data=clients_data,
    eval_data=eval_data,
    model=model,
    loss_fn=loss_fn,
)

In [4]:
import threading

client_threads = []
for i, client in enumerate(clients):
    client_threads.append(threading.Thread(target=run_client, args=(i, client)))
    client_threads[-1].start()
    
master_thread = threading.Thread(target=run_master, args=(master,))
master_thread.start()

master_thread.join()
for t in client_threads:
    t.join()
    


Master: 0.6651252508163452
Master: 0.6199489831924438
Master: 0.5952244400978088
Master: 0.5693540573120117
Master: 0.5498557090759277
Master: 0.5322489738464355
Master: 0.5173061490058899
Master: 0.5041072964668274
Master: 0.49246183037757874
Master: 0.48207324743270874
Master: 0.47275015711784363
Master: 0.46433162689208984
Master: 0.45669037103652954
Master: 0.4497222602367401
Master: 0.4433417022228241
Master: 0.43747738003730774
Master: 0.43206942081451416
Master: 0.42706695199012756
Master: 0.4224267601966858
Master: 0.4181116223335266
Master: 0.4140894114971161
Master: 0.41033199429512024
Master: 0.4068150520324707
Master: 0.4035170376300812
Master: 0.40041905641555786
Master: 0.3975042700767517
Master: 0.39475783705711365
Master: 0.39216628670692444
Master: 0.3897179067134857
Master: 0.38740184903144836
Master: 0.38520845770835876
Master: 0.3831290006637573
Master: 0.38115569949150085
Master: 0.37928125262260437
Master: 0.37749913334846497
Master: 0.37580347061157227
Master: 0.